In [1]:
import datetime
import re

from bs4 import BeautifulSoup
import requests

In [2]:
def get_category_urls(category_url):
    i = 0; result = []
    while True:
        i += 1
        page_url = category_url+'&p='+str(i)
        soup = BeautifulSoup(requests.get(category_url).text,'html.parser')
        urls = ['http://www.setn.com'+soup.select("ol a")[i].attrs['href'] for i in range(len(soup.select("ol a")))]
        result.extend(urls)
        if len(urls)==0 or i>5:
            break
    
    return result

In [3]:
url = 'http://www.setn.com/ViewAll.aspx?pagegroupid=4'
urls = get_category_urls(url)
urls

['http://www.setn.com/ColumnNews.aspx?NewsID=118349&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118309&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118346&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118283&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118321&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118311&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118308&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118298&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118266&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118279&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118243&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118263&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118253&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118171&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118199&PageGroupID=4',
 'http://www.setn.com/News.aspx?NewsID=118247&PageGroupID=4',
 '

In [4]:
def parser_page(url):
    """
    parser page method 
    主要用於抓取新聞detail頁面資訊

    page_data = parser_page(news detail url)
    """
    soup = BeautifulSoup(requests.get(url).text,'html.parser')
    search = lambda typ,seed: soup.find_all("meta", attrs={typ : seed})[0].attrs['content']
    inputs = [("property","og:url"),("property","og:title"),("name","pubdate"),("name","Keywords"),("name","section")]
    
    parse = [search(typ,seed) for typ,seed in inputs]
    post_time = datetime.datetime(*map(lambda x: int(x),re.findall('[0-9]+',parse[2])))
    
    article = soup.select("article div > p")
    content = "".join([article[i].text for i in range(1,len(article))])
    journalist = article[0].text
    
    fb_url = "http://www.facebook.com/v2.3/plugins/like.php?app_id=1385360291698338&"+ \
            "href=http%3A%2F%2Fwww.setn.com%2FNews.aspx%3FNewsID%3D" 
    News_id = re.findall("NewsID=([0-9]+)",url)[0]
    
    soup_fb = BeautifulSoup(requests.get(fb_url+News_id).text,'html.parser')
    like_str = "".join(re.findall('[0-9]',soup_fb.find_all("span", attrs={"class": "hidden_elem"})[0].text))
    fb_like = int(like_str) if len(like_str)>0 else None
    
    return {
        "url": parse[0],
        "source_press": None,
        "title": parse[1],
        "post_time": post_time,
        "journalist": journalist,
        "content": content,
        "compare": None,
        "keyword": parse[3].split(','),
        "fb_like": fb_like,
        "fb_share": None,
        "category": parse[4].split(',')
    }

In [5]:
parse_content = [parser_page(urls[i]) for i in range(5)]
parse_content

[{'category': ['名家'],
  'compare': None,
  'content': '很多愛穿高跟鞋的女生，一定都有遇過買了高跟鞋卻發現不合腳的情形，太緊的鞋子穿起來，整隻腳就像要炸裂一般，發生這種情形時，該怎麼辦呢？小編來教大家一個國外流傳的「冰撐法」，簡單幾步驟就可以穿得下鞋子囉！\xa0 \xa0步驟一：首先呢，需要的材料很簡單，拿兩個夾鏈袋，裝入4分之1的水。▲圖／翻攝自YouTube\xa0\xa0\xa0步驟二：接著，將水袋塞進高跟鞋中。▲圖／翻攝自YouTube\xa0\xa0\xa0步驟三：再把高跟鞋放進冰箱冷凍庫一個晚上...▲圖／翻攝自YouTube隔天把高跟鞋拿出來之後，就可以順利穿進去了！其實，這招是利用水變成冰，體積會變大的原理，讓水袋將鞋子撐大。下次不妨試試這個方法，讓穿不下的高跟鞋起死回生吧！▲圖／翻攝自YouTube▼影片取自YouTube，如遭刪除請見諒。',
  'fb_like': None,
  'fb_share': None,
  'journalist': '文／珍奶不加珍珠',
  'keyword': ['高跟鞋',
   '鞋子',
   '哇潮',
   '合腳',
   '冰撐法',
   '三立',
   '三立新聞',
   '三立新聞台',
   '三立財經台',
   '新聞台',
   '財經台'],
  'post_time': datetime.datetime(2016, 1, 14, 17, 19),
  'source_press': None,
  'title': '【哇潮】高跟鞋不合腳？這樣做就能順利穿進去 | 生活 | 三立新聞網  SETN.COM',
  'url': 'http://www.setn.com/ColumnNews.aspx?NewsID=118349'},
 {'category': ['生活'],
  'compare': None,
  'content': '新北市一位劉先生買新車加購萬元配備，升級音響配備、抬頭顯示器和自動鎖裝置，沒想到交車時車商卻忘記裝上去，音響配備還是便宜款式，他氣得投訴平面媒體。但車商出面喊冤，坦承漏裝其中兩樣配備是疏失，但是音響系統當初核定時，消費者也親自確認，事後發現有更高級款式，居然回頭求償，還獅子大開